In [1]:
import os
import json
import traceback
import pandas as pd

In [2]:
import langchain
%pip install langchain_google_genai

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
KEY=os.getenv("GOOGLE_API_KEY")


In [6]:
llm=ChatGoogleGenerativeAI(google_api_key=KEY,model="gemini-2.5-flash",temperature=0.5, max_output_tokens=1500)

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here", 
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here", 
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here", 
        },
        "correct": "correct answer",
    },
}

In [9]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [11]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)

C:\Users\GrimReaper\AppData\Local\Temp\ipykernel_972\1958558231.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)


In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
and if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits
the students ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"], template=TEMPLATE)

In [14]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chains=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text","number","subject","tone","response_json"],
                                         output_variables=["quiz","review"], verbose=True,)


In [16]:
file_path=r"E:\AI\data.txt"

In [17]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [18]:
print(TEXT)

Unrated
In the standard non-ranked mode, the match is played as best of 25, with the first team to win 13 rounds winning the match. The attacking team has a bomb-type device called the Spike. They must deliver and activate the Spike on one of the multiple specified locations (bomb sites). If the attacking team successfully protects the activated Spike for 45 seconds it detonates, destroying everything in a specific area, and they receive a point.[5] If the defending team can deactivate the spike, or the 100-second round timer expires without the attacking team activating the spike, the defending team receives a point.[15] If all the members of a team are eliminated before the spike is activated, or if all members of the defending team are eliminated after the spike is activated, the opposing team earns a point.[5] If both teams win 12 rounds, sudden death occurs, in which the winning team of that round wins the match, differing from overtime for competitive matches. Additionally, if af

In [19]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [ ]:
NUMBER=5
SUBJECT="Nepal"
TONE="simple"

In [21]:

response=generate_evaluate_chains(
        {
            "text":TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Unrated
In the standard non-ranked mode, the match is played as best of 25, with the first team to win 13 rounds winning the match. The attacking team has a bomb-type device called the Spike. They must deliver and activate the Spike on one of the multiple specified locations (bomb sites). If the attacking team successfully protects the activated Spike for 45 seconds it detonates, destroying everything in a specific area, and they receive a point.[5] If the defending team can deactivate the spike, or the 100-second round timer expires without the attacking team activating the spike, the defending team receives a point.[15] If all the members of a team are eliminated before the spike is activated, or if all members of the defending team are eliminated after the spike is activated, the opposing team earns a point.[5] If both teams win 12 rounds, sudden death occurs, in which the win

C:\Users\GrimReaper\AppData\Local\Temp\ipykernel_972\1981401647.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chains(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Text:Unrated
In the standard non-ranked mode, the match is played as best of 25, with the first team to win 13 rounds winning the match. The attacking team has a bomb-type device called the Spike. They must deliver and activate the Spike on one of the multiple specified locations (bomb sites). If the attacking team successfully protects the activated Spike for 45 seconds it detonates, destroying everything in a specific area, and they receive a point.[5] If the defending team can deactivate the spike, or the 100-second round timer expires without the attacking team activating the spike, the defending team receives a point.[15] If all the members of a team are eliminated before the spike is activated, or if all members of the defending team are eliminated after the spike is activated, the opposing team earns a point.[5] If both teams win 12 rounds, sudden death occurs, in which the winning team of that round

In [22]:
quiz=response.get("quiz")

In [23]:
import ast

# remove code fences
import re
cleaned = re.sub(r"^```json\n", "", quiz)
cleaned = re.sub(r"\n```$", "", cleaned)

data = ast.literal_eval(cleaned)   # handles single quotes, dict-style
print(data.keys())


dict_keys(['1', '2', '3', '4', '5'])


In [24]:
quiz=data

In [25]:
quiz_table_data=[]
for key,value in quiz.items():
    mcq= value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq,"Choices": options, "Correct": correct})

In [26]:
quiz_table_data

[{'MCQ': 'In the standard Unrated mode, how many rounds does a team need to win to win the match?',
  'Choices': 'a: 7 rounds | b: 9 rounds | c: 13 rounds | d: 25 rounds',
  'Correct': 'c'},
 {'MCQ': 'What is a special rule about the Spike in Spike Rush mode?',
  'Choices': 'a: Only one player on the attacking team carries a Spike. | b: Defending players can also carry a Spike. | c: All players on the attacking team carry a Spike. | d: Spikes detonate instantly in Spike Rush.',
  'Correct': 'c'},
 {'MCQ': 'How many rounds does a team need to win to win a Swiftplay match, and about how long does a Swiftplay game usually take?',
  'Choices': 'a: 4 rounds, 40 minutes | b: 5 rounds, 15 minutes | c: 13 rounds, 15 minutes | d: 7 rounds, 40 minutes',
  'Correct': 'b'},
 {'MCQ': 'In an Unrated match, how many votes are needed for a team to successfully surrender?',
  'Choices': 'a: 3 votes | b: 4 votes | c: 5 votes | d: 12 votes',
  'Correct': 'b'},
 {'MCQ': 'Which of these is true about ultim

In [27]:
quiz=pd.DataFrame(quiz_table_data)

In [28]:
quiz.to_csv("machinelearning.csv",index=False)